In [12]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import os

os.chdir(os.getcwd().replace('notebooks',''))

# Changing the default cache directory of the models
# os.environ['HF_HOME'] = r'J:\HuggingFace\cache' # For newer versions of transformers (>= v5)
# os.environ['TRANSFORMERS_CACHE'] = r'J:\HuggingFace\cache' # For older versions of transformers

In [13]:
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")

GPU: NVIDIA GeForce RTX 3060 Ti is available.


In [14]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-medium"
# model_id = "openai/whisper-large-v3" 
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=False, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device
)

In [15]:
recog = pipe(r'audio_samples/2963847.wav', generate_kwargs={"language": "portuguese"})

/home/gbrlmoraes/git_reps/VoiceToInsight/.venv/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


In [16]:
recog

{'text': ' Olá, você ligou para Brico Bread. Estamos há mais de 25 anos levando o melhor da padaria para dentro de sua casa. Pão crocante e quentinho a qualquer hora. Digite 1 se você quer falar Dois, pós-vendas. Três, cadastro. Quatro, qualidade. Cinco, compras. Seis, recursos humanos. Sete, logística. Oito, tecnologia. Você conhece o nosso portifólio de produtos? Acesse www.bricobrad.com É, Gabrico, boa tarde, Raquel. Boa tarde, Raquel. Meu nome é Viviane. Eu falo aqui da Totos Matriz, tudo bem? Tudo bem. Eu gostaria, por favor, de falar com o Valtor Pereira? Tá, só um momento. Tudo bem. É da? Da Totos. Tá, um momento, tudo bem. E da? Da TOTUS. Viviane, oi, pode falar. O que seria? A gente está entrando em contato com vocês para fazer um acompanhamento em relação à marca TOTUS que vocês utilizam na empresa. Tá, é para acompanhar? Acompanhamento em relação ao sistema que vocês utilizam, só para verificar se está tudo ok com a utilização e se vocês estão sendo bem atendidos. Ah, tá bom

In [17]:
# Creating a file with the complete text without the timestamps
with open(r'data/complete_texts/text_2963847.txt', 'w', encoding = 'utf8') as f:
    f.write(recog['text'])

# Creating a file with the text segmented by timestamps
with open(r'data/timestamp_texts/timestamp_2963847.txt', 'w', encoding = 'utf8') as f:

    timestamp_text = ''
    for timestamp in recog['chunks']:
        timestamp_text += f'{timestamp}\n'

    f.write(timestamp_text)